<a href="https://colab.research.google.com/github/Amir-Kermanshahani/AI/blob/main/DRL/Unit1_Lunar_Lander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing package builders
!apt install swig cmake

In [ ]:
# Installing stable-baselines3==2.0.0a5, swig, gymnasium[box2d], huggingface_sb3
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

In [ ]:
# Installing virtual screen libraries
!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
# Resetting runtime to ensure the pakcages work correctly
import os
os.kill(os.getpid(), 9)

In [ ]:
# Virtual display configuration
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
#Importing packages
import gymnasium as gym

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [ ]:
# Inspecting the observations space

"""
The observation is a vector of size 8, where each value contains different information about the lander:
- Horizontal pad coordinate (x)
- Vertical pad coordinate (y)
- Horizontal speed (x)
- Vertical speed (y)
- Angle
- Angular speed
- If the left leg contact point has touched the land (boolean)
- If the right leg contact point has touched the land (boolean)
"""

env = gym.make("LunarLander-v2")
env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample())


_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [-40.408607   -25.957434    -4.380131    -1.7921468   -1.1122987
  -0.9469312    0.09854485   0.8645833 ]


In [ ]:
#Inspecting action space

"""
The action space is discrete with 4 actions available :
- Action 0: Do nothing,
- Action 1: Fire left orientation engine,
- Action 2: Fire the main engine,
- Action 3: Fire right orientation engine.
"""

print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 0


In [ ]:
"""
After every step a reward is granted. The total reward of an episode is the **sum of the rewards for all the steps within that episode**.

For each step, the reward:

- Is increased/decreased the closer/further the lander is to the landing pad.
- Is increased/decreased the slower/faster the lander is moving.
- Is decreased the more the lander is tilted (angle not horizontal).
- Is increased by 10 points for each leg that is in contact with the ground.
- Is decreased by 0.03 points each frame a side engine is firing.
- Is decreased by 0.3 points each frame the main engine is firing.

The episode receive an additional reward of -100 or +100 points for crashing or landing safely respectively.

An episode is considered a solution if it scores at least 200 points.
"""

In [ ]:
# Creating the vectorized environment
env = make_vec_env('LunarLander-v2', n_envs=16)

In [ ]:
# Defining the model
# Fine PPO parameters at: https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html#parameters

model = DQN(
    policy = 'MlpPolicy',
    env = env,
    # n_steps = 1024,
    batch_size = 64,
    # n_epochs = 4,
    gamma = 0.999,
    # gae_lambda = 0.98,
    # ent_coef = 0.01,
    verbose=1)

Using cuda device


#### Solution

In [ ]:
# Training the model for 1,000,000 timesteps
model.learn(total_timesteps=1000000)
# Saving the model
model_name = "ppo-LunarLander-v2"
model.save(model_name)

Streaming output truncated to the last 5000 lines.
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 1.44     |
|    n_updates        | 437      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 143      |
|    ep_rew_mean      | -558     |
|    exploration_rate | 0.255    |
| time/               |          |
|    episodes         | 760      |
|    fps              | 3725     |
|    time_elapsed     | 21       |
|    total_timesteps  | 78400    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 1.3      |
|    n_updates        | 443      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 145      |
|    ep_rew_mean      | -552     |
|    exploration_rate | 0.253    |
| time/               |          |
|    episodes         | 764      |
|   

In [ ]:
# Evaluating the agent
# Find evaluation helper docs at: https://stable-baselines3.readthedocs.io/en/master/common/evaluation.html#module-stable_baselines3.common.evaluation

eval_env = Monitor(gym.make("LunarLander-v2", render_mode='rgb_array'))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")